# Finetune Stable Diffusion 集成化环境训练
如果你是在autodl上的机器可以直接使用封装好的镜像创建实例，开箱即用  
如果是本地或者其他服务器上也可以使用，需要手动安装一些pip包

## 注意
本项目仅供用于学习、测试人工智能技术使用  
请勿用于训练生成不良或侵权图片内容  



### 本文件为notebook在线运行版
具体详细的教程和参数说明请在根目录下教程.md 文件中查看。  
在notebook中执行linux命令，需要前面加个!(感叹号)  
代码块前如果有个[*]，表示正在运行该步骤，并不是卡住了  
如果遇到问题可到b站主页找该教程对应训练演示的视频：https://space.bilibili.com/291593914 

### 首先，进入工作文件夹
记得先把项目文件夹移动到autodl-tmp目录下 

In [ ]:
%cd /root/autodl-tmp/dreambooth-for-diffusion

### 然后，把环境缓存（权重）移到系统目录下

In [ ]:
!rm -rf /root/.cache 
!mv /root/env-cache /root/.cache 

### 那么，开始你的AIGC训练之旅吧
欢迎在Github上关注我们的组织, 持续维护更多AIGC领域好用的工具！https://github.com/idpen  
—— 造梦笔   

# 一、准备数据集
该部分请参考教程.md文件中的详细内容自行上传并处理你的数据集  
./datasets/test中为16张仅供于学习测试的样本数据，便于你了解以下代码的用处  


### 批量图片超分、降噪、去模糊
这一步是可以跳过的。  
### 只针对质量差的小图使用!大图超分会爆显存
这里提供了一份批量超分处理，针对本身图源比较差而且尺寸比较小的图片处理。

针对写实图片使用：

In [ ]:
!python tools/make_upscale.py \
        -n RealESRGAN_x4plus \
        -i ./datasets/test \
        -o ./datasets/test_upscale  

针对写实图片人物的面部提升：

In [ ]:
!python tools/make_upscale.py \
        -n RealESRGAN_x4plus \
        -i ./datasets/test \
        -o ./datasets/test_upscale \
        --face_enhance 

针对动漫图使用：

In [ ]:
!python tools/make_upscale.py \
        -n RealESRGAN_x4plus_anime_6B \
        -i ./datasets/test \
        -o ./datasets/test_upscale  

### 批量处理背景 + 自适应调整尺寸

./datasets/test是原始图片数据文件夹，请上传你的图片数据并进行更换  
./datasets/test2是输出处理后图片路径  

--max_size 可以把宽高的最大边缩放到指定尺寸内（按比例缩放），如--max_size 1024  
--png2black 对透明底的png图处理成黑色底  
--png2white 对透明底的png图处理成白色底  

In [ ]:
!python tools/handle_images.py ./datasets/test_upscale ./datasets/test2 --png2black --max_size 768  

### 批量标注图片
使用deepdanbooru生成tags标注文件。（仅针对纯二次元类图片效果较好，其他风格请手动标注）  
./datasets/test2中是需要打标注的图片数据，请按需更换为自己的路径 

动漫图片打标（danbooru tags）：

In [ ]:
# 该步根据需要标注文件数量不同，需要运行一段时间  
!python tools/label_images_with_tags.py \
        --path ./datasets/test2 \ 
        --threshold 0.75 # 阈值 

写实图片打标 (自然句子描述)：

In [ ]:
# 该步根据需要标注文件数量不同，需要运行一段时间  
!python tools/label_images_with_caption.py \
        -f ./datasets/test2 \ 
        -d "cuda" \
        -c  "ViT-L-14/openai" \
        -m "fast" 

### 批量arb裁剪 
可以有效处理不规则图片的一种裁剪方式  
这里的离线arb处理可以帮助我们提前准备好数据集，不用每次训练时都做在线处理。

In [ ]:
!python tools/make_arb.py ./datasets/test2 \
    -o ./datasets/test_arb \
    --max-height 1024 \
    --max-width 1024 \
    --min-height 256 \
    --min-width 256 

# 二、准备预训练模型

## 转换检查点文件为diffusers官方权重
输出的文件在dreambooth-for-diffusion/model下  
--checkpoint_path  权重文件路径   
--vae_path vae权重路径（转换时直接把单独的vae权重合并进去，使用总权重时不用再单独挂载vae文件 ） 

--from_safetensors 是否是safetensors格式文件

### 以下参数只有转换sd2.1时才需要
--prediction_type="v-prediction"  
--upcast_attention=True

转换safetensors格式权重：

In [ ]:
# 该步需要运行大约一分钟 
!python tools/all2diffusers.py \
    --checkpoint_path=./ckpt_models/Basil_mix_fixed.safetensors \
    --vae_path=./ckpt_models/animevae.pt \
    --dump_path=./base_model \
    --original_config_file=./ckpt_models/model-v1.yaml \
    --scheduler_type="ddim" \
    --from_safetensors 

转换ckpt格式权重：

In [ ]:
# 该步需要运行大约一分钟 
!python tools/all2diffusers.py \
    --checkpoint_path=./ckpt_models/xxx.ckpt \
    --vae_path=./ckpt_models/animevae.pt \
    --dump_path=./base_model \
    --original_config_file=./ckpt_models/model-v1.yaml \
    --scheduler_type="ddim" 

# 三、进行Finetune训练
以下训练脚本会自动帮你启动tensorboard日志监控进程，入口可参考: https://www.autodl.com/docs/tensorboard/  
使用tensorboard面板可以帮助分析loss在不同step的总体下降情况  
如果你嫌输出太长，可以在以下命令每一行后加一句 &> log.txt, 会把输出都扔到这个文件中 
```
!sh train_style.sh &> log.txt
```
本代码包环境已在A5000、3090测试通过，如果你在某些机器上运行遇到问题可以尝试卸载编译的xformers
```
!pip uninstall xformers
```

### 如果需要训练特定人、事物： 
（推荐准备3~5张风格统一、特定对象的图片）  
请打开train_object.sh具体修改里面的参数

In [ ]:
# 大约十分钟后才会在tensorboard有日志（因为前十分钟在生成同类别伪图）
!sh train_object.sh 

### 如果要训练画风： 
（推荐准备3000+张图片，包含尽可能的多样性，数据决定训练出的模型质量）  
请打开train_object具体修改里面的参数  
实测速度1000步大概8分钟  

In [ ]:
# 正常训练立刻就可以在tensorboard看到日志
!sh train_style.sh  

后台训练法请参考教程.md中的内容

省钱训练法（训练成功后自动关机，适合步数很大且夜晚训练的场景）

In [ ]:
!sh back_train.sh

### 拓展：训练Textual inversion

In [ ]:
!sh train_textual_inversion.sh

## LoRA融合部分
可以将权重之间进行加权合并

### 仅融合unet_lora到总权重
./output_merged 为得到的总权重

In [ ]:
!lora_add ./base_model ./new_model/epoch_6_step_0/lora_weights/lora_e6_s0.pt ./output_merged 0.8 --mode upl

### 融合unet_lora和text_encder_lora到总权重
./output_merged 为得到的总权重

In [ ]:
!lora_add ./base_model ./new_model/epoch_6_step_0/lora_weights/lora_e6_s0.pt ./output_merged 0.8 --mode upl with_text_lora

### 拓展：加权两个LoRA权重
这里给出一个伪代码例子，最后得到的xxx3.pt = xxx1.pt * 0.3 + xxx2.pt * 0.7

In [ ]:
!lora_add --path_1 xxx1.pt --path_2 xxx2.pt --mode lpl --alpha 0.7 --output_path xxx3.pt

## 总权重融合部分 
关于模型融合部分可以参考这篇文章：https://note.com/kohya_ss/n/n9a485a066d5b

### 分层融合
如--input_blocks "0:0.3"，0是input层序号，0.3是第二个权重的占比，多层描述之间以英文逗号分隔，层内以英文冒号分隔。  
这里的权重比例指的是第二个权重的占比。  

如果不需要融合middle_blocks，可以直接删除--middle_blocks这一行。  
支持safetensors格式和ckpt格式权重的融合与保存（根据文件格式自动判断）。

In [ ]:
!python tools/merge_unet_blocks.py ./ckpt_models/xxx1.safetensors ./ckpt_models/xxx2.safetensors \
        --input_blocks "0:0.5, 1:0.5, 2:0.5, 3:0.6, 4:0.5, 5:0.5, 6:0.5, 7:0.5, 8:0.5, 9:0.5, 10:0.5, 11:0.5" \
        --middle_blocks "0:0.5, 1:0.5, 2:0.6" \
        --output_blocks "0:0.5, 1:0.5, 2:0.5, 3:0.6, 4:0.5, 5:0.5, 6:0.5, 7:0.5, 8:0.5, 9:0.5, 10:0.5, 11:0.5" \
        --out "0:0.5, 2:0.3" \
        --time_embed "0:0.5, 2:0.3" \
        --dump_path ./ckpt_models/merged.ckpt  

以上这段等价于：(即只为特别层指定融合时的比例，其他层融合时共用基础比例--base_alpha 0.5)

In [ ]:
!python tools/merge_unet_blocks.py ./ckpt_models/xxx1.safetensors ./ckpt_models/xxx2.ckpt \
        --base_alpha 0.5 \
        --input_blocks "3:0.6" \
        --middle_blocks "2:0.6" \
        --output_blocks "3:0.6," \
        --out "2:0.3" \
        --time_embed "2:0.3" \
        --dump_path ./ckpt_models/merged.safetensors  

你也可以，以相同的比例合并整个unet层：

In [ ]:
!python tools/merge_unet_blocks.py ./ckpt_models/xxx1.safetensors ./ckpt_models/xxx2.ckpt \
        --base_alpha 0.5 \
        --dump_path ./ckpt_models/merged.safetensors   

## 测试总权重效果
打开dreambooth-for-diffusion/test_model.py文件修改其中的model_path和prompt，然后执行以下测试  
会生成一张图片 在左侧test-1、2、3.png

In [ ]:
# 大约5~10s 
!python test_model.py

# 四、权重格式转换 

### 转换diffusers总权重为ckpt检查点 
输出的文件在dreambooth-for-diffusion/ckpt_models/中，名为newModel.ckpt

原始保存：

In [ ]:
!python tools/diffusers2ckpt.py ./output_merged ./ckpt_models/newModel.ckpt 

以下代码添加--half 保存float16半精度，权重文件大小会减半（约2g），效果基本一致

In [ ]:
!python tools/diffusers2ckpt.py ./model2 ./ckpt_models/newModel_half.ckpt --half

### 转换ckpt为safetensors格式

In [ ]:
!python tools/ckpt2safetensors.py -i ./ckpt_models/newModel_half.ckpt -o ./ckpt_models/newModel_half.safetensors

### 转换safetensors为ckpt格式

In [ ]:
!python tools/safetensors2ckpt.py -i ./ckpt_models/newModel_half.safetensors -o ./ckpt_models/newModel_2.ckpt 

下载总权重文件，去玩吧~

有问题可以进XDiffusion QQ Group：455521885  

# 五、其它内容

### 记得定期清理不需要的中间权重和文件，不然容易导致空间满
大部分问题已在教程.md中详细记录，也包含其他非autodl机器手动部署该训练一体化封装代码包的步骤

In [ ]:
# 清理文件的示例
!rm -rf ./model* # 删除当前目录model文件/文件夹
!rm -rf ./new_model/* # 删除当前目录所有new_开头的模型文件夹
# !rm -rf ./datasets/test2 #删除datasets中的test2数据集 

linux压缩一个文件夹为单个文件包的命令：
```
!zip xx.zip -r ./xxx
```
解压一个包到文件夹：
```
!unzip xx.zip -d xxx
```
或许你在上传、下载数据集时会用到。

其他linux基础命令：https://www.autodl.com/docs/linux/

关于文件上传下载的提速可查看官网文档推荐的几种方式：https://www.autodl.com/docs/scp/